# Siamese Network

In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K

In [2]:
num_classes = 10
epochs = 20

In [3]:
# def create_pairs(x, digit_indices):
#     pairs = []
#     labels = []
#     nums = []
#     n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
#     for d in range(n):
#         for i in range(2):
#             z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
#             pairs += [[x[z1], x[z2]]]
#             nums += [[d, d]]
#             inc = random.randrange(1, num_classes)
#             dn = (d + inc) % num_classes
#             z1, z2 = digit_indices[d][i], digit_indices[dn][i]
#             pairs += [[x[z1], x[z2]]]
#             labels += [1, 0]
#             nums += [[d, dn]]
#     return np.array(pairs), np.array(labels), np.array(nums)

In [12]:
import numpy as np
import random

def create_pairs(x, digit_indices, num_classes):
    pairs = []
    labels = []
    nums = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            if i + 1 < len(digit_indices[d]):
                z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
                pairs += [[x[z1], x[z2]]]
                labels += [1]
                nums += [[d, d]]
                inc = random.randrange(1, num_classes)
                dn = (d + inc) % num_classes
                if i < len(digit_indices[dn]):
                    z1, z2 = digit_indices[d][i], digit_indices[dn][i]
                    pairs += [[x[z1], x[z2]]]
                    labels += [0]
                    nums += [[d, dn]]
    return np.array(pairs), np.array(labels), np.array(nums)


In [13]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [14]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [15]:
def constrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [16]:
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    x  = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(2, activation='relu')(x)
    return Model(input, x)

In [17]:
def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

def compute_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [18]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0
input_shape = x_train.shape[1:]

In [19]:
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y, tr_nums = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y, te_nums = create_pairs(x_test, digit_indices)

TypeError: create_pairs() missing 1 required positional argument: 'num_classes'

In [21]:
base_network = create_base_network(input_shape)
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
processed_a = base_network(input_a)
processed_b = base_network(input_b)
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
model = Model([input_a, input_b], distance)
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='siamese_network.png')

TypeError: Inputs to a layer should be tensors. Got '<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x0000025FD9F270A0>>' (of type <class 'method'>) as input for layer 'flatten_1'.

In [22]:
tr_y = tf.cast(tr_y, dtype='float32')
te_y = tf.cast(te_y, dtype='float32')

NameError: name 'tr_y' is not defined

In [23]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

NameError: name 'model' is not defined

In [ ]:
H = model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y, validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y), batch_size=128, epochs=10, verbose=1)

In [ ]:
intermediate_layer_model = Model([input=a.input_b],[processed_a, processed_b])
tr_intermediate_output = intermediate_layer_model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
te_intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
tr_intermediate_output_arr = np.array(tr_intermediate_output)
te_intermediate_output_arr = np.array(te_intermediate_output)

In [24]:
tr_nums_flat = []
for row in tr_nums:
    for x in row:
        tr_nums_flat.append(x)

tr_features_flat = []
for i in range(len(tr_intermediate_output_arr[0])):
    tr_features_flat.append(tr_intermediate_output_arr[0][i])
    tr_features_flat.append(tr_intermediate_output_arr[1][i])

tr_nums_flat = np.array(tr_nums_flat)
tr_features_flat = np.array(tr_features_flat)

NameError: name 'tr_nums' is not defined

In [ ]:
te_features_flat = []
for i in range(len(te_intermediate_output_arr[0])):
    te_features_flat.append(te_intermediate_output_arr[0][i])
te_features_flat = np.array(te_intermediate_output_arr[1][i])

te_nums_flat = np.array(te_nums_flat)
te_features_flat = np.array(te_features_flat)